In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *

In [ ]:
## Load the catalog
cat = load_cat_photoz_gaia('csv/minijpas.flambda_photoz_gaia.csv')

In [ ]:
tcurves = load_tcurves(load_filter_tags())

In [ ]:
# Define the array of the filters we are going to use // gSDSS band
nb_ind_arr = [8, 9, 10, 11, 12, 13, 14, 15]
bb_ind = -3

In [ ]:
# Function to drop sources
def make_masks(cat):
    # Mask sources with high photoz odds
    mask_pz_odds = cat['odds'] < 0.98
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_sn = np.abs(cat['parallax'] / cat['parallax_err'])
    pmra_sn = np.abs(cat['pmra'] / cat['pmra_err'])
    pmdec_sn = np.abs(cat['pmdec'] / cat['pmdec_err'])
    mask_pmotion = (
        (np.sqrt(parallax_sn**2 + pmra_sn**2 + pmdec_sn**2) < 27.**0.5)
        | ( np.isnan(parallax_sn) | np.isnan(pmra_sn) | np.isnan(pmdec_sn) )
    )
    
    mask_total = mask_pz_odds & mask_pmotion
    return mask_total

In [ ]:
mask = make_masks(cat)

In [ ]:
pm_flx = cat['flx'][mask] * 1e-19
pm_err = cat['flx_err'][mask] * pm_flx
nb_ind = 11
fwhm_nb = nb_fwhm(tcurves, nb_ind, True)
w_central = np.array(central_wavelength(load_tcurves(load_filter_tags())))
N_nb = 10 # Number of nb on each side of the central one
line, cf = nbex_cont_estimate(pm_flx, pm_err, nb_ind, w_central, N_nb, 50, fwhm_nb)

In [ ]:
len(np.where(line)[0])

In [ ]:
bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
for i in range(10000):
    if ~line[i]: continue
        
    pm = pm_flx[i]
    errors = pm_err[i]
    cont_fit = cf[i]
    
    fig, ax = plt.subplots()
    ax.scatter(w_central[:-3], pm[:-3])
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], fmt='none')
    ax.scatter(w_central[nb_ind], pm[nb_ind], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4], xerr=bb_fwhm[-4]/2, fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    x_fit = np.linspace(4000, 6000, 1000)
    y_fit = cont_fit[1] + cont_fit[0]*x_fit
    ax.plot(x_fit,y_fit, c='dimgray', linestyle='dashed')

    plt.show()